# Import module

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version : ',torch.__version__,'Device :',DEVICE)

Using PyTorch version :  1.11.0 Device : cpu


# Parameter

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10

# Data Load

In [4]:
train_dataset = datasets.MNIST(root = "../MNIST/data/MNIST",
                                train = True,
                                download = True,
                                transform = transforms.ToTensor())
test_dataset = datasets.MNIST(root = "../MNIST/data/MNIST",
                                train = False,
                                transform = transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = False)                                           

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../MNIST/data/MNIST\MNIST\raw\train-images-idx3-ubyte.gz to ../MNIST/data/MNIST\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../MNIST/data/MNIST\MNIST\raw\train-labels-idx1-ubyte.gz to ../MNIST/data/MNIST\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../MNIST/data/MNIST\MNIST\raw\t10k-images-idx3-ubyte.gz to ../MNIST/data/MNIST\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../MNIST/data/MNIST\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../MNIST/data/MNIST\MNIST\raw

